# Что сделал:

- добавил выход из jarvis_loop при final_answer + защита от зацикливания
- дополнил docs разной инфой классик мл
- добавил RAG(простенький класс для загрузки доков, сохранения;  использование при ответе по флагу) -> точность стабильно выросла с 0.9 до 1.0
- добавил timeout в ColabExecutor


- Analyst Agent (решает: нужна команда)
- Command Agent (генерирует код)
- Colab Runtime (исполняет)

In [ ]:
import uuid
import json
import faiss
import os
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

def llm_chat(system, user):
    completion = client.chat.completions.create(
        model="openai/gpt-5-mini",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
    )
    return completion.choices[0].message.content.strip()



/Users/riter/anaconda3/envs/autorag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class MemoryAgent:
    def __init__(self, dim=384):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.index = faiss.IndexFlatL2(dim)
        self.store = {}

    def add(self, text: str):
        emb = self.model.encode([text])
        self.index.add(emb)
        self.store[self.index.ntotal - 1] = text

    def search(self, query: str, k=3):
        if self.index.ntotal == 0:
            return []
        q_emb = self.model.encode([query])
        _, idx = self.index.search(q_emb, k)
        return [self.store[i] for i in idx[0] if i in self.store]

In [4]:
class DocumentChunker:
    """Разбивает длинные документы на чанки с перекрытием."""
    
    def __init__(self, chunk_size=500, overlap=50):
        self.chunk_size = chunk_size
        self.overlap = overlap
    
    def chunk(self, text: str, source: str = None) -> list:
        """Разбивает текст на чанки с метаданными.
        
        Args:
            text: Исходный текст для разбиения
            source: Источник документа (путь к файлу, URL и т.д.)
        
        Returns:
            Список словарей с ключами: text, source, chunk_id
        """
        if len(text) <= self.chunk_size:
            return [{"text": text, "source": source, "chunk_id": 0}]
        
        chunks = []
        start = 0
        chunk_id = 0
        
        while start < len(text):
            end = start + self.chunk_size
            chunk_text = text[start:end]
            
            # Пытаемся разбить по границе предложения или слова
            if end < len(text):
                # Ищем последнюю точку или пробел
                last_period = chunk_text.rfind('. ')
                last_space = chunk_text.rfind(' ')
                
                if last_period > self.chunk_size // 2:
                    end = start + last_period + 1
                    chunk_text = text[start:end]
                elif last_space > self.chunk_size // 2:
                    end = start + last_space
                    chunk_text = text[start:end]
            
            chunks.append({
                "text": chunk_text.strip(),
                "source": source,
                "chunk_id": chunk_id
            })
            
            start = end - self.overlap if end - self.overlap > start else end
            chunk_id += 1
        
        return chunks
    
    def chunk_documents(self, documents: list, source_key: str = None) -> list:
        """Разбивает список документов на чанки.
        
        Args:
            documents: Список строк или словарей с текстом
            source_key: Ключ для получения источника из словаря
        
        Returns:
            Список чанков
        """
        all_chunks = []
        for i, doc in enumerate(documents):
            if isinstance(doc, dict):
                text = doc.get("text", str(doc))
                source = doc.get(source_key) if source_key else f"doc_{i}"
            else:
                text = str(doc)
                source = f"doc_{i}"
            
            chunks = self.chunk(text, source)
            all_chunks.extend(chunks)
        
        return all_chunks

In [5]:
import pickle
import numpy as np

class RetrieverAgent:
    def __init__(self, docs=None, dim=384):
        """
        Args:
            docs: Опциональный список документов для инициализации.
                  Может быть списком строк или словарей с ключом 'text'.
            dim: Размерность эмбеддингов
        """
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.index = faiss.IndexFlatL2(dim)
        self.documents = []
        self.dim = dim
        
        if docs:
            self.add_documents(docs)
    
    def add_documents(self, docs: list):
        """Добавляет документы в индекс.
        
        Args:
            docs: Список строк или словарей с ключом 'text'
        """
        # Нормализуем формат документов
        normalized_docs = []
        for i, doc in enumerate(docs):
            if isinstance(doc, dict):
                normalized_docs.append({
                    "text": doc.get("text", str(doc)),
                    "source": doc.get("source", f"doc_{len(self.documents) + i}"),
                    "metadata": doc.get("metadata", {})
                })
            else:
                normalized_docs.append({
                    "text": str(doc),
                    "source": f"doc_{len(self.documents) + i}",
                    "metadata": {}
                })
        
        # Создаём эмбеддинги
        texts = [d["text"] for d in normalized_docs]
        embeddings = self.model.encode(texts)
        
        # Добавляем в индекс
        self.index.add(np.array(embeddings).astype('float32'))
        self.documents.extend(normalized_docs)
        
        print(f"Добавлено {len(normalized_docs)} документов. Всего: {len(self.documents)}")
    
    def add_from_file(self, filepath: str, chunker: DocumentChunker = None):
        """Загружает документы из файла.
        
        Args:
            filepath: Путь к текстовому файлу
            chunker: Опциональный DocumentChunker для разбиения на чанки
        """
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
        
        if chunker:
            chunks = chunker.chunk(text, source=filepath)
        else:
            chunks = [{"text": text, "source": filepath}]
        
        self.add_documents(chunks)
    
    def search(self, query: str, k: int = 3) -> list:
        """Поиск релевантных документов.
        
        Args:
            query: Поисковый запрос
            k: Количество результатов
        
        Returns:
            Список словарей с ключами: text, source, metadata, score
        """
        if self.index.ntotal == 0:
            return []
        
        # Ограничиваем k количеством документов
        k = min(k, self.index.ntotal)
        
        q_emb = self.model.encode([query])
        distances, indices = self.index.search(np.array(q_emb).astype('float32'), k)
        
        results = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.documents) and idx >= 0:
                doc = self.documents[idx].copy()
                doc["score"] = float(distances[0][i])
                results.append(doc)
        
        return results
    
    def search_texts(self, query: str, k: int = 3) -> list:
        """Поиск релевантных документов, возвращает только тексты.
        
        Args:
            query: Поисковый запрос
            k: Количество результатов
        
        Returns:
            Список строк с текстами документов
        """
        results = self.search(query, k)
        return [r["text"] for r in results]
    
    def save(self, path: str):
        """Сохраняет индекс и документы на диск.
        
        Args:
            path: Базовый путь (без расширения)
        """
        faiss.write_index(self.index, f"{path}.index")
        with open(f"{path}.docs", "wb") as f:
            pickle.dump(self.documents, f)
        print(f"Сохранено в {path}.index и {path}.docs")
    
    def load(self, path: str):
        """Загружает индекс и документы с диска.
        
        Args:
            path: Базовый путь (без расширения)
        """
        self.index = faiss.read_index(f"{path}.index")
        with open(f"{path}.docs", "rb") as f:
            self.documents = pickle.load(f)
        print(f"Загружено {len(self.documents)} документов из {path}")


In [6]:
import signal

class ColabExecutor:
    def __init__(self, timeout: int = 30):
        self.timeout = timeout
    
    def _timeout_handler(self, signum, frame):
        raise TimeoutError(f"Execution timed out after {self.timeout} seconds")
    
    def run(self, code: str):
        # Устанавливаем таймаут
        old_handler = signal.signal(signal.SIGALRM, self._timeout_handler)
        signal.alarm(self.timeout)
        
        try:
            local_env = {}
            exec(code, {}, local_env)
            return {
                "status": "success",
                "output": str(local_env)
            }
        except TimeoutError as e:
            return {
                "status": "error",
                "traceback": str(e)
            }
        except Exception as e:
            return {
                "status": "error",
                "traceback": str(e)
            }
        finally:
            signal.alarm(0)
            signal.signal(signal.SIGALRM, old_handler)

In [7]:
def command_agent(task: str, memory: MemoryAgent, executor: ColabExecutor):
    # 1. генерация кода
    code = client.chat.completions.create(
        model="openai/gpt-5-mini",
        messages=[
            {"role": "system", "content": "Сгенерируй Python-код. Только код."},
            {"role": "user", "content": task},
        ],
    ).choices[0].message.content

    # 2. выполнение
    result = executor.run(code)

    # 3. автодебаг
    retries = 2
    while result["status"] == "error" and retries > 0:
        code = client.chat.completions.create(
            model="openai/gpt-5-mini",
            messages=[
                {"role": "system", "content": "Исправь ошибку и верни только код."},
                {"role": "user", "content": f"Код:\n{code}\nОшибка:\n{result['traceback']}"}
            ]
        ).choices[0].message.content

        result = executor.run(code)
        retries -= 1

    # 4. ревью
    review = client.chat.completions.create(
        model="openai/gpt-5-mini",
        messages=[
            {"role": "system", "content": "Сделай краткое техническое ревью."},
            {"role": "user", "content": f"Код:\n{code}\nРезультат:\n{result}"}
        ]
    ).choices[0].message.content

    memory.add(f"Task: {task}\nCode:\n{code}\nResult:\n{result}")

    return f"{result}\n\nREVIEW:\n{review}"

In [8]:
functions = [
    {
        "name": "search_docs",
        "description": "Поиск по документации",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string"}
            },
            "required": ["query"]
        }
    },
    {
        "name": "run_code",
        "description": "Сгенерировать и выполнить код",
        "parameters": {
            "type": "object",
            "properties": {
                "task": {"type": "string"}
            },
            "required": ["task"]
        }
    },
    {
        "name": "final_answer",
        "description": "Финальный ответ пользователю",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {"type": "string"}
            },
            "required": ["answer"]
        }
    }
]

In [9]:
def dispatch(name, args, retriever, memory, executor):
    if name == "search_docs":
        query = args.get("query") or args.get("q") or str(args)
        results = retriever.search(query, k=5)
        # Форматируем результаты для LLM
        formatted = []
        for r in results:
            text = r.get("text", str(r))
            source = r.get("source", "unknown")
            formatted.append(f"[{source}] {text}")
        return "\n".join(formatted) if formatted else "Документы не найдены"

    if name == "run_code":
        task = args.get("task") or args.get("code") or args.get("description") or str(args)
        return command_agent(task, memory, executor)

    if name == "final_answer":
        answer = args.get("answer") or args.get("text") or args.get("response") or args.get("message") or str(args)
        return answer

    raise ValueError(f"Unknown action: {name}")

In [10]:
import json

def jarvis_loop(user_input, retriever, memory, executor, use_rag_context=True, verbose=False):
    # Получаем релевантную документацию из RAG
    if use_rag_context:
        relevant_docs = retriever.search(user_input, k=5)
        doc_context = "\n".join([f"- {d['text']}" for d in relevant_docs])
    else:
        doc_context = ""
    
    # Получаем релевантную историю из памяти
    history = memory.search(user_input, k=2)
    history_context = "\n".join(history) if history else ""
    
    # Формируем системный промпт с контекстом
    system_content = "Ты JARVIS — AI ассистент программиста.\n\n"
    
    if doc_context:
        system_content += f"Релевантная документация:\n{doc_context}\n\n"
    
    if history_context:
        system_content += f"История взаимодействий:\n{history_context}\n\n"
    
    system_content += (
        "Используй документацию для генерации правильного кода.\n"
        "Если нужно выполнить задачу, ответь строго в JSON формате:\n"
        '{"action": "run_code", "args": {"task": "описание задачи"}}\n'
        '{"action": "search_docs", "args": {"query": "поисковый запрос"}}\n'
        '{"action": "final_answer", "args": {"answer": "финальный ответ"}}'
    )
    
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_input}
    ]

    iterations = 0
    while iterations < 10:
        iterations += 1
        
        response = client.chat.completions.create(
            model="openai/gpt-5-mini",
            messages=messages
        )

        msg_text = response.choices[0].message.content.strip()

        # Пытаемся разобрать JSON
        try:
            msg_json = json.loads(msg_text)
            action = msg_json.get("action")
            args = msg_json.get("args", {})
        except Exception:
            # Если LLM не вернул JSON, выдаём как финальный ответ
            memory.add(f"USER: {user_input}\nASSISTANT: {msg_text}")
            return msg_text

        # Диспетчер
        result = dispatch(action, args, retriever, memory, executor)

        # добавляем результат в контекст для следующего шага
        messages.append({"role": "assistant", "content": msg_text})
        messages.append({"role": "user", "content": f"Результат {action}: {result}"})

        if verbose:
            print(f"Action: {action}")
            print(f"Result: {result[:200]}..." if len(str(result)) > 200 else f"Result: {result}")
            print()

        if action == "final_answer":
            memory.add(f"USER: {user_input}\nASSISTANT: {result}")
            return result
    max_iterations_msg = "Превышено максимальное количество итераций. Попробуйте переформулировать запрос."
    return max_iterations_msg

In [11]:
docs = [
    # sklearn - классификация
    "RandomForestClassifier — ансамблевый метод на основе деревьев решений. Параметры: n_estimators (количество деревьев), max_depth (глубина), random_state. Пример: RandomForestClassifier(n_estimators=100, random_state=42)",
    "LogisticRegression — линейная модель для бинарной и мультиклассовой классификации. Параметры: C (регуляризация), solver, max_iter.",
    "SVC (Support Vector Classifier) — классификатор на основе опорных векторов. Параметры: kernel (linear, rbf, poly), C, gamma.",
    "GradientBoostingClassifier — градиентный бустинг для классификации. Параметры: n_estimators, learning_rate, max_depth.",
    "XGBClassifier — оптимизированный градиентный бустинг из библиотеки xgboost. Быстрее sklearn GradientBoosting. Пример: XGBClassifier(n_estimators=100, learning_rate=0.1)",
    "DecisionTreeClassifier — дерево решений для классификации. Параметры: max_depth, min_samples_split, criterion (gini/entropy).",
    "KNeighborsClassifier — метод k ближайших соседей. Параметры: n_neighbors, weights (uniform/distance), metric.",
    
    # sklearn - регрессия
    "LinearRegression — линейная регрессия методом наименьших квадратов. fit(X, y), predict(X), coef_, intercept_.",
    "Ridge и Lasso — регрессия с L2 и L1 регуляризацией соответственно. Параметр alpha контролирует силу регуляризации.",
    "RandomForestRegressor — ансамблевая регрессия на деревьях. Аналогичен классификатору, но для непрерывных целевых переменных.",
    "GradientBoostingRegressor — градиентный бустинг для регрессии. Параметры: n_estimators, learning_rate, max_depth.",
    
    # sklearn - метрики
    "accuracy_score — доля правильных предсказаний. from sklearn.metrics import accuracy_score; accuracy_score(y_true, y_pred)",
    "precision_score, recall_score, f1_score — метрики для классификации. Параметр average='macro'/'micro'/'weighted' для мультикласса.",
    "confusion_matrix — матрица ошибок классификации. Строки — истинные классы, столбцы — предсказанные.",
    "mean_squared_error, mean_absolute_error — метрики регрессии. MSE штрафует большие ошибки сильнее.",
    "r2_score — коэффициент детерминации R^2. Показывает долю объяснённой дисперсии. 1.0 — идеальная модель.",
    "classification_report — сводка precision, recall, f1 по классам. from sklearn.metrics import classification_report",
    "roc_auc_score — площадь под ROC-кривой. Метрика качества бинарной классификации.",
    
    # sklearn - препроцессинг
    "train_test_split — разделение на train/test. Параметры: test_size, random_state, stratify для сохранения пропорций классов.",
    "StandardScaler — стандартизация (z-score): (x - mean) / std. fit_transform(X_train), transform(X_test).",
    "MinMaxScaler — нормализация в диапазон [0, 1]. Чувствителен к выбросам.",
    "LabelEncoder — преобразование категориальных меток в числа 0, 1, 2, ...",
    "OneHotEncoder — one-hot кодирование категориальных признаков. sparse_output=False для плотной матрицы.",
    "cross_val_score — кросс-валидация. Параметры: estimator, X, y, cv (число фолдов). Возвращает массив скоров.",
    "GridSearchCV — поиск лучших гиперпараметров по сетке. param_grid — словарь параметров.",
    "Pipeline — объединение препроцессинга и модели. Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression())])",
    
    # sklearn - кластеризация
    "KMeans — кластеризация методом k-средних. Параметры: n_clusters, random_state. fit_predict(X).",
    "DBSCAN — плотностная кластеризация. Параметры: eps (радиус), min_samples. Находит кластеры произвольной формы.",
    "AgglomerativeClustering — иерархическая кластеризация. Параметры: n_clusters, linkage (ward/complete/average).",
    
    # sklearn - снижение размерности
    "PCA — метод главных компонент. Параметры: n_components. fit_transform(X) для снижения размерности.",
    "TSNE — t-SNE для визуализации высокоразмерных данных в 2D/3D. Параметры: n_components, perplexity.",
    
    # pandas
    "pandas DataFrame — основная структура данных. df.head(), df.info(), df.describe() для просмотра.",
    "df.dropna() — удаление строк с пропусками. df.fillna(value) — заполнение пропусков.",
    "df.groupby('column').agg({'col2': 'mean'}) — группировка и агрегация.",
    "pd.read_csv('file.csv') — загрузка CSV. Параметры: sep, encoding, index_col.",
    "df.merge(df2, on='key') — соединение таблиц. how='left'/'right'/'inner'/'outer'.",
    "df.apply(func, axis=1) — применение функции к строкам. axis=0 для столбцов.",
    "df.loc[condition] — фильтрация по условию. df.loc[df['col'] > 5]",
    "df.value_counts() — подсчёт уникальных значений. df['col'].value_counts()",
    "df.pivot_table() — сводная таблица. values, index, columns, aggfunc.",
    
    # numpy
    "numpy array — эффективный многомерный массив. np.array([1, 2, 3]), np.zeros((3, 4)), np.ones((2, 2)).",
    "np.random.seed(42) — фиксация генератора случайных чисел для воспроизводимости.",
    "np.mean(), np.std(), np.sum() — агрегатные функции. axis=0 по столбцам, axis=1 по строкам.",
    "np.reshape() — изменение формы массива. arr.reshape(-1, 1) для преобразования в столбец.",
    "np.concatenate(), np.vstack(), np.hstack() — объединение массивов.",
    "np.where(condition, x, y) — условный выбор элементов.",
    
    # datasets
    "load_iris() — датасет ирисов Фишера. 150 примеров, 4 признака, 3 класса. from sklearn.datasets import load_iris",
    "load_digits() — рукописные цифры 8x8. 1797 примеров, 64 признака, 10 классов.",
    "fetch_california_housing() — цены на жильё в Калифорнии. Задача регрессии. 20640 примеров.",
    "make_classification(), make_regression() — генерация синтетических данных для тестирования моделей.",
    "load_breast_cancer() — диагностика рака груди. 569 примеров, 30 признаков, 2 класса.",
    
    # matplotlib / визуализация
    "plt.figure(figsize=(10, 6)) — создание фигуры с размером.",
    "plt.plot(x, y), plt.scatter(x, y), plt.bar(x, y), plt.hist(data) — типы графиков.",
    "plt.xlabel(), plt.ylabel(), plt.title(), plt.legend() — подписи.",
    "plt.savefig('plot.png', dpi=300) — сохранение графика.",
    "plt.subplot(rows, cols, index) — создание сетки графиков.",
    "seaborn.heatmap(data) — тепловая карта. annot=True для отображения значений.",
    
    # FAISS
    "FAISS — библиотека Facebook для быстрого поиска ближайших соседей. IndexFlatL2 для точного поиска.",
    "faiss.IndexFlatL2(dim) — создание индекса. index.add(vectors), index.search(query, k).",
    "faiss.write_index(index, path) — сохранение индекса. faiss.read_index(path) — загрузка.",
    
    # общие советы
    "Всегда фиксируйте random_state для воспроизводимости результатов.",
    "Используйте stratify в train_test_split для сбалансированного разбиения при несбалансированных классах.",
    "Масштабируйте признаки (StandardScaler/MinMaxScaler) перед обучением линейных моделей и SVM.",
    "Для деревьев и ансамблей (RandomForest, GradientBoosting) масштабирование не требуется.",
]

In [12]:
retriever = RetrieverAgent(docs)
memory = MemoryAgent()
executor = ColabExecutor()

Добавлено 64 документов. Всего: 64


In [13]:
answer = jarvis_loop(
    "Обучи RandomForest на iris и выведи accuracy",
    retriever,
    memory,
    executor,
    verbose=True
)

print(answer)

1.0
Action: run_code
Result: {'status': 'success', 'output': '{\'np\': <module \'numpy\' from \'/Users/riter/anaconda3/envs/autorag/lib/python3.11/site-packages/numpy/__init__.py\'>, \'load_iris\': <function load_iris at 0x30c5fe...

Action: final_answer
Result: Ниже — короткий исправленный пример с учётом замечаний (stratify, n_jobs, cross_val_score, classification_report):

from sklearn.datasets import load_iris
from sklearn.model_selection import train_tes...

Ниже — короткий исправленный пример с учётом замечаний (stratify, n_jobs, cross_val_score, classification_report):

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Загрузка и разбиение (сохранение пропорций классов)
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size